In [101]:
import tensorflow as tf
# coding=gbk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

##定义常量
rnn_unit = 10       # hidden layer units
input_size = 7
output_size = 1
lr=0.0006 # 学习率
testtime = 280

# ——————————————————导入数据——————————————————————
f = open('./rnn/stock_predict/dataset/399300.csv', encoding= 'gbk')
df = pd.read_csv(f, dtype = {'change': np.float64,
 'close': np.float64,
 'high': np.float64,
 'label': np.float64,
 'low': np.float64,
 'money': np.float64,
 'open': np.float64,
 'volume': np.float64})# 读入股票数据

data = df.iloc[:,2:10].values  # 取第3-10列


# 获取训练集
def get_train_data(batch_size=60,time_step=20,train_begin=0,train_end=testtime):
    batch_index=[]
    data_train=data[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #标准化
    train_x,train_y=[],[]   #训练集 
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:7]
        y=normalized_train_data[i:i+time_step,7,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y

In [120]:
def get_test_data(time_step,test_begin):
    data_test=data[test_begin:]
    mean=np.mean(data_test,axis=0)
    std=np.std(data_test,axis=0)
    normalized_test_data=(data_test-mean)/std  #标准化
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size个sample 
    test_x,test_y=[],[]
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:7]
        y=normalized_test_data[i*time_step:(i+1)*time_step,7]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:7]).tolist())
    test_y.extend((normalized_test_data[(i+1)*time_step:,7]).tolist())
    return mean,std,test_x,test_y

In [130]:
np.array(test_x[0]).shape

(20, 7)

In [132]:
def lstm(X):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']
    input=tf.reshape(X,[-1,input_size]) #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

In [144]:
weights={
         'in':tf.Variable(tf.random_normal([input_size,rnn_unit])),
         'out':tf.Variable(tf.random_normal([rnn_unit,1]))
        }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'out':tf.Variable(tf.constant(0.1,shape=[1,]))
       }

X=tf.placeholder(tf.float32, shape=[None,15,7])

In [149]:
with tf.variable_scope('lstm', reuse=tf.AUTO_REUSE):
    pred, fis = lstm(X)

In [151]:
pred

<tf.Tensor 'lstm_4/add_1:0' shape=(?, 1) dtype=float32>

In [152]:
fis

LSTMStateTuple(c=<tf.Tensor 'lstm_4/rnn/while/Exit_3:0' shape=(?, 10) dtype=float32>, h=<tf.Tensor 'lstm_4/rnn/while/Exit_4:0' shape=(?, 10) dtype=float32>)

In [155]:
Y=tf.placeholder(tf.float32, shape=[None,15,1])
loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))

In [157]:
loss

<tf.Tensor 'Mean_6:0' shape=() dtype=float32>

In [128]:
test_y

[-0.9660465662090862,
 -1.0111286439734608,
 -1.0547338132624942,
 -1.0883485031648126,
 -1.04117282066666,
 -1.1277000495430727,
 -1.1096008049941066,
 -0.9872008291460145,
 -1.149581697810468,
 -1.14043245537116,
 -1.1948472031799169,
 -1.2806185838216269,
 -1.241899546426381,
 -1.2278104088297284,
 -1.2032026468455408,
 -1.191115400180129,
 -1.0991707318641946,
 -1.0270710328903474,
 -1.0702113080768652,
 -1.1017735063292997,
 -0.9092599097140256,
 -0.5955670795878748,
 -0.6435681863088616,
 -0.6600070947774137,
 -0.6870848043402736,
 -0.685396005355624,
 -0.7746588355835762,
 -0.758735421936181,
 -0.7965784343899528,
 -0.8626471602107543,
 -0.8685990697409229,
 -0.8664137512046077,
 -0.8531120872918078,
 -0.8031249023641557,
 -0.8860373424026406,
 -0.8463853542574463,
 -0.8287826292601473,
 -0.756218036183783,
 -0.761935917390236,
 -0.7658985861688223,
 -0.8339565527412062,
 -1.009288042832887,
 -1.0164701823350197,
 -1.0847938026802693,
 -1.0076435194770488,
 -1.0257364389361863,


In [121]:
mean,std,test_x,test_y = get_test_data(20, 2800)

In [122]:
np.array(test_x).shape

(31,)

In [123]:
np.array(test_y).shape

(601,)

In [103]:
batch_index,train_x,train_y = get_train_data()

In [105]:
batch_index

[0, 60, 120, 180, 240, 260]

In [110]:
np.array(train_x).shape

(260, 20, 7)

In [111]:
train_x[0]

[[0.9727694109511192,
  0.772951739477028,
  0.8799082437670324,
  0.8594423288595858,
  -1.0427779113127957,
  -0.8973702083777944,
  -1.3237459892205412],
 [0.7550464105793429,
  0.9346474296128757,
  0.8669187939668055,
  0.9016650743596374,
  -1.0912420654425243,
  -0.8666326279443376,
  0.7480051514290708],
 [0.9489998036322909,
  0.7792408246614314,
  0.8744127073130912,
  0.8432155482360367,
  -1.2283525951747372,
  -1.0589013623249208,
  -0.7323839274167591],
 [0.7793112180506464,
  0.7923155017553242,
  0.865753074112938,
  0.8750067919066642,
  -1.0616143587452636,
  -0.8007948360995188,
  0.05080835675795345],
 [0.7910309549925698,
  0.956493725516594,
  0.8654200112975479,
  0.8460303979360413,
  -1.3102366572776865,
  -1.1088787776362978,
  0.7587641257147912],
 [0.9631955413365919,
  1.0104474563091081,
  1.0536005019931503,
  0.9385892792479152,
  -1.3007698473230564,
  -1.1274831858283056,
  0.23973899740649088],
 [1.0038019538395908,
  1.00399286888301,
  1.02312525438

In [112]:
np.array(train_y).shape

(260, 20, 1)